# Generic Mapping Types

In [4]:
from collections import abc

In [5]:
my_dict = {}

In [7]:
assert isinstance(my_dict, abc.Mapping) == True

## What is hashable?

**Hash tables** are the engines behind Python’s **high** performance dicts.

"An object is hashable if it has a hash value which never changes during its lifetime (it needs a `__hash__()` method), and can be compared to other objects (it needs an `__eq__()` method). Hashable objects which compare equal must have the same hash value.""

**“All of Python’s immutable built-in objects are hashable, but not all hashable objects are immutable”**

In [3]:
my_name = "Carla"
hash(my_name)

6904093625884498950

In [15]:
my_tuple = (20,30)
hash(my_tuple)

3713067559093172781

In [4]:
l = [1, 2, 3]
hash(l)

TypeError: unhashable type: 'list'

## Different ways to create a dictionary

In [6]:
a = dict(one=1, two=2, three=3)

b = {'one': 1, 'two': 2, 'three': 3}

c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))  

d = dict([('two', 2), ('one', 1), ('three', 3)])  

e = dict({'three': 3, 'one': 1, 'two': 2}) 
assert a == b == c == d == e

In [16]:
# zip to generate dictionaries
keys = ['a', 'b', 'c']
values = [1, 2, 3]
d3 = dict(zip(keys, values))
d3

{'a': 1, 'b': 2, 'c': 3}

# dict comprehensions

Produces a `key:value` pair from any iterable.

In [9]:
DIAL_CODES = [
	(86, 'China'),
	(91, 'India'),
	(1, 'United States'),
	(62, 'Indonesia'),
	(55, 'Brazil'),
	(92, 'Pakistan'),
	(880, 'Bangladesh'),
	(234, 'Nigeria'),
	(7, 'Russia'),
	(81, 'Japan'),
]

In [10]:
country_code = {country: code for code, country in DIAL_CODES}

In [11]:
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [12]:
{code: country.upper() for country, code in country_code.items() if code < 66}   

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

# Overview of Common Mapping Methods

- dict
- defaultdict
- OrderedDict

## how `d.update(m, [**kargs])` works - *duck typing*

**Duck typing**: é um estilo de codificação de linguagens dinamicamente tipadas onde o tipo de uma variável não importa, contanto que seu comportamento seja o desejado. 

"se anda como pato, nada como um pato e faz quack como um pato, então provavelmente é um pato"

In [49]:
# Dictionary with three items  
Dictionary1 = { 'ORDER_FINISHED': 'finalizado', 'ORDER_STARTED': 'começado', } 
m = { 'ORDER_STARTED': 'iniciado' } 

In [50]:
# Dictionary before Updation 
print(Dictionary1) 

Original Dictionary:
{'ORDER_FINISHED': 'finalizado', 'ORDER_STARTED': 'começado'}


### **1)** `update` first checks whether `m` has a keys method and, if it does, assumes it is a mapping

In [51]:
assert (m.keys is not None) == True

In [52]:
# update the value of key 'B' 
Dictionary1.update(m)

In [63]:
print(Dictionary1) 

{'ORDER_FINISHED': 'finalizado', 'ORDER_STARTED': 'iniciado', 'ORDER_CANCELLED': 'cancelado', 'ORDER_AWAITING_PAYMENT': 'aguardando pagamento'}


### **2)** Otherwise, update falls back to iterating over `m`, assuming its items are `(key, value)` pairs.

In [59]:
m = (
    ('ORDER_STARTED', 'iniciado'), 
    ("ORDER_CANCELLED", "cancelado"),
    ("ORDER_AWAITING_PAYMENT","aguardando pagamento")
    )

In [60]:
Dictionary1.update(m)

In [62]:
print(Dictionary1) 

{'ORDER_FINISHED': 'finalizado', 'ORDER_STARTED': 'iniciado', 'ORDER_CANCELLED': 'cancelado', 'ORDER_AWAITING_PAYMENT': 'aguardando pagamento'}


## Handling missing keys with `d.setdefault(k, [default])`

- fail-fast philosophy

In [69]:
doggo = {'name': 'Lassie','breed': "border collie"}

In [73]:
print(doggo.nome)

AttributeError: 'dict' object has no attribute 'nome'

###  `d.get(k, default)` is an alternative to `d[k] `

In [68]:
n = doggo.get('nome', )

In [69]:
assert n == None

Se queremos atribuir um valor para uma chave que não existe, o `.get`pode ser ineficiente.

In [70]:
x = doggo.setdefault('is_castrated', False)
print(x)

False


In [71]:
doggo

{'name': 'Lassie', 'breed': 'border collie', 'is_castrated': False}

### defaultdict: Another Take on Missing Keys

In [120]:
from collections import defaultdict

In [121]:
# default_factory é a função anônima
ice_cream = defaultdict(lambda: 'Creme')

In [122]:
ice_cream['Carla'] = 'Chocolate'
ice_cream['Yara'] = 'Morango'
ice_cream['Joe']
print(ice_cream)

defaultdict(<function <lambda> at 0x11100a560>, {'Carla': 'Chocolate', 'Yara': 'Morango', 'Joe': 'Creme'})


In [123]:
ice_cream.get('Joe')

'Creme'

In [124]:
food_list = 'spam spam spam spam spam spam eggs spam'.split()

In [125]:
# default_factory é o valor inteiro 0
food_count = defaultdict(int) # default value of int is 0
print(food_count)

defaultdict(<class 'int'>, {})


In [126]:
for food in food_list:
    food_count[food] += 1
    
print(food_count)

defaultdict(<class 'int'>, {'spam': 7, 'eggs': 1})


### The `__missing__` Method